In [ ]:
import re
import pickle
from janome.tokenizer import Tokenizer
import collections
from gensim.models import word2vec
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument


with open("bot.txt", mode="r", encoding="utf-8") as f:  # ファイルの読み込み
    bot_original = f.read()

bot = re.sub("[\n]", "。", bot_original) # | と全角半角スペース、「」と改行の削除
bot = re.sub("[｜ 　]", "", bot) # | と全角半角スペース、「」と改行の削除

# 終わり記号の重複を除去
bot = bot.replace('！。', '！')
bot = bot.replace('？。', '？')
bot = bot.replace('♪。', '♪')
bot = bot.replace('？*', '？')
bot = bot.replace('♪？', '♪')
bot = bot.replace('。*', '。')

seperator = "。"  # 。をセパレータに指定
bot_list = re.split('[。！？♪]', bot)  # セパレーターを使って文章をリストに分割する
bot_list.pop() # 最後の要素は空の文字列になるので、削除
bot_list.pop(0)
bot_list = [x+seperator for x in bot_list]  # 文章の最後に。を追加

t = Tokenizer()

bot_words = []
for sentence in bot_list:
    bot_words.append(t.tokenize(sentence, wakati=True))   # 文章ごとに単語に分割し、リストに格納
    
with open('bot_words.pickle', mode='wb') as f:  # pickleに保存
    pickle.dump(bot_words, f)

# 読み込み用
# with open('bot_words.pickle', mode='rb') as f:
#     bot_words = pickle.load(f)

In [ ]:
# size : 中間層のニューロン数
# min_count : この値以下の出現回数の単語を無視
# window : 対象単語を中心とした前後の単語数
# iter : epochs数
# sg : skip-gramを使うかどうか 0:CBOW 1:skip-gram
model = word2vec.Word2Vec(bot_words,
                          size=100,
                          min_count=5,
                          window=5,
                          iter=20,
                          sg = 0)


In [ ]:
print(model.wv.vectors.shape)  # 分散表現の形状
print(model.wv.vectors)  # 分散表現
print(len(model.wv.index2word))  # 語彙の数
print(model.wv.index2word[:10])  # 最初の10単語

In [ ]:
print(model.wv.most_similar("おはよう"))  # 最も似ている単語

In [ ]:
tagged_documents = []
for i, sentence in enumerate(bot_words):
    tagged_documents.append(TaggedDocument(sentence, [i]))  # TaggedDocument型のオブジェクトをリストに格納

# size：分散表現の次元数
# window：対象単語を中心とした前後の単語数
# min_count：学習に使う単語の最低出現回数
# epochs:epochs数
# dm：学習モデル=DBOW（デフォルトはdm=1で、学習モデルはDM）
model = Doc2Vec(documents=tagged_documents,
                vector_size=100,
                min_count=5,
                window=5,
                epochs=20,
                dm=0)

In [ ]:
print(bot_words[0])  # 最初の文章を表示
print(model.docvecs[0])  # 最初の文章のベクトル

In [ ]:
for p in model.docvecs.most_similar(0):
    print(bot_words[p[0]])